# NDArray: A NDim, Compressed Data Container

NDArray objects let users perform different operations with NDArray arrays like setting, copying or slicing them. In this section, we are going to see how to create and manipulate a NDArray array in a simple way.


In [ ]:
import numpy as np

import blosc2

## Creating an array
Let's start creating an 2-D array with 100M elements filled with arange.


In [ ]:
%time array = blosc2.arange(10_000 * 10_000, shape=(10_000, 10_000))
print(array.info)

Note that all the compression and decompression parameters, as well as the chunks and blocks shapes are set to the default.

## Reading and modifying data
We can read and modify NDArray arrays using NumPy array as data source.

In [ ]:
array[0]

In [ ]:
array[0, :] = blosc2.zeros(10000, dtype=array.dtype)
array[:, 0] = blosc2.ones(10000, dtype=array.dtype)

In [ ]:
array[0, 0]

In [ ]:
array[0, :]

In [ ]:
array[:, 0]

## Enlarging the array
Existing arrays can be enlarged.

In [ ]:
array.resize((10_001, 10_000))
print(array.shape)
array[10_000, :] = 1
array[10_000, :]

Enlarging is a fast operation because data is chunked, and we just have to add more chunks into the array, so no need to copy all the data to a new location (as in the case of a NumPy array, which requires a full copy of the data).

You can also shrink the array.

In [ ]:
array.resize((9_000, 10_000))
print(array.shape)
print(array[8_999])  # This works
#array[9_000]  # This will raise an exception

## Persistent data
We can use the `save()` method to store the array on disk.  This is very useful when you have a large array that you want to keep around but do not need to access all the time.


In [ ]:
array.save("array_tutorial.b2nd", mode="w") #, contiguous=True)
!ls -lh array_tutorial.b2nd


For arrays, it is usual to use the `.b2nd` extension.

Also, when we create a NDArray array, we can specify where it will be stored, and no memory will be used at all. Indeed, we can specify all the compression/decompression and other storage parameters.

In [ ]:
array1 = blosc2.full(
    (1000, 1000),
    fill_value=b"pepe",
    chunks=(100, 100),
    blocks=(50, 50),
    urlpath="array1_tutorial.b2nd",
    mode="w",
)
!ls -lh array1_tutorial.b2nd

This time we have set the chunks and blocks shapes.

Now, let's reopen our original array.

In [ ]:
array2 = blosc2.open("array_tutorial.b2nd")
print(array2.info)

And make sure that they are the same.

In [ ]:
np.all(array2[:] == array[:])

## Compression params
Let's see how to make a copy of a NDArray array, while changing its compression parameters in an easy way.

In [ ]:
cparams = blosc2.CParams(
    codec=blosc2.Codec.LZ4,
    clevel=9,
    filters=[blosc2.Filter.BITSHUFFLE],
    filters_meta=[0],
)

array2 = array.copy(chunks=(500, 10_000), blocks=(50, 10_000), cparams=cparams)
print(array2.info)

## Metadata handling
We can add metadata to a NDArray array via the `vlmeta` property, which is a dictionary-like object.


In [ ]:
print(array.vlmeta)
array.vlmeta["info1"] = "This is an example"
array.vlmeta["info2"] = "of user meta handling"
array.vlmeta[:]  # this returns all the metadata as a dictionary

You can store any kind of data in the `vlmeta` metalayer, as long as it is serializable with msgpack.  This is a very flexible way to store metadata in a Blosc2 container.

In [ ]:
array.vlmeta["info3"] = {"a": 1, "b": 2}
array.vlmeta

Variable length metadata can be deleted:

In [ ]:
del array.vlmeta["info1"]
array.vlmeta

This is useful to store metadata that is not known at the time of creation of the container, or that can be updated or deleted at any time.

## Creating a NDArray from a NumPy array

Let's create a NDArray from a NumPy array using the `asarray` constructor:

In [ ]:
shape = (100, 100, 100)
dtype = np.float64
nparray = np.linspace(0, 100, np.prod(shape), dtype=dtype).reshape(shape)
b2array = blosc2.asarray(nparray)
print(b2array.info)
b2array[0, 0, :4]

## Building a NDArray from an iterator

Finally, let's see how you can create a NDArray filled with data from an iterator, store it into a file, and reopen it. Let's create a structured array with 3 fields and 1 million of elements.

In [ ]:
N = 1000_000
rng = np.random.default_rng()
it = ((-x + 1, x - 2, rng.normal()) for x in range(N))
%time sa = blosc2.fromiter(it, dtype='i4,f4,f8', shape=(N,), urlpath="sa-1M_tutorial1.b2nd", mode="w")
!ls -lh sa-1M_tutorial1.b2nd
sa2 = blosc2.open("sa-1M_tutorial1.b2nd")
sa2.info

In this case we have created a structured array with 3 fields and 1 million of elements.  The file size is around 7 MB, so the compression ratio is a bit better than 2x.  This is because the data is random and it is hard to compress.

Now that we have seen how to create and manipulate NDArray arrays, let's move to the next tutorial where we will see how to create and use LazyArray arrays.

That's all for this part!  Questions?